## Quickstart: Búsqueda Vectorial para Principiantes

#### Inicializar el cliente

Antes de comenzar a realizar consultas o insertar vectores, debemos inicializar el cliente de Qdrant.

In [1]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333")

Asegúrate de que tu instancia de Qdrant esté corriendo localmente o cambia la URL/puerto según tu configuración.

#### Crear una colección

Una colección en Qdrant es como una base de datos para tus vectores. Define el nombre y el tipo de vectores que vas a almacenar (tamaño y distancia).

In [2]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT)
)

True

#### Agregar vectores

Una vez creada la colección, puedes insertar vectores junto con metadatos (payload). Cada vector debe tener un identificador único (`id`), una lista de valores numéricos (`vector`) y un diccionario opcional de información adicional (`payload`).

In [3]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


### Ejecutar una consulta

Hagamos una pregunta básica: ¿Cuáles de nuestros vectores almacenados son más similares al vector de consulta `[0.2, 0.1, 0.9, 0.7]`?

In [4]:
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

print(*search_result, sep='\n')

id=4 version=0 score=1.362 payload=None vector=None shard_key=None order_value=None
id=1 version=0 score=1.273 payload=None vector=None shard_key=None order_value=None
id=3 version=0 score=1.208 payload=None vector=None shard_key=None order_value=None


### Agregar un filtro
Podemos refinar aún más los resultados aplicando un filtro sobre los metadatos (payload). Por ejemplo, veamos los resultados más cercanos que además incluyan "London" en su payload:

In [5]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
).points

print(*search_result, sep='\n')

id=2 version=0 score=0.871 payload={'city': 'London'} vector=None shard_key=None order_value=None
